In [24]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import operator
import gc
import time

import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import string
import re

import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

from keras.layers import add, Dense, Input, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras import backend as K
from keras import models
from keras.engine.topology import Layer
from keras.utils import np_utils

In [45]:
train = pd.read_csv('./data/BEST&MOST200/train-best200.csv')
dev = pd.read_csv('./data/BEST&MOST200/dev-best200.csv')
test = pd.read_csv('./data/BEST&MOST200/test-best200.csv')

In [26]:
print('Train shape:', train.shape)
print('Dev shape  :', dev.shape)
print('Test shape :', test.shape)

Train shape: (96585, 457)
Dev shape  : (34028, 457)
Test shape : (32977, 457)


In [27]:
def merge_same_user(df):
    df_class = df[['user-id', 'class']].groupby('user-id').apply(lambda x: x['class'].mode()).reset_index()
    df_class = df_class.rename(columns={0: 'class'})
    df_new = df.groupby('user-id').sum().reset_index()
    df_new = pd.merge(df_new, df_class, on='user-id')
    
    return df_new

In [28]:
train_per_user = merge_same_user(train)
dev_per_user = merge_same_user(dev)

train_per_user = pd.concat([train_per_user, dev_per_user], axis=0)
test_per_user = merge_same_user(test)

In [29]:
print('Train per user shape:', train_per_user.shape)
print('Test per user shape :', test_per_user.shape)

Train per user shape: (3190, 457)
Test per user shape : (802, 457)


In [31]:
X = train_per_user.drop(['tweet-id', 'user-id', 'class'], axis=1)
X_test = test_per_user.drop(['tweet-id', 'user-id', 'class'], axis=1)

# mmsc = MinMaxScaler()
# X = mmsc.fit_transform(X)
# X_test = mmsc.transform(X_test)

ss = StandardScaler()
X = ss.fit_transform(X)
X_test = ss.transform(X_test)

y = train_per_user['class']
encoder = LabelEncoder()
encoder.fit(y)
y_one_hot = encoder.transform(y)
y_one_hot = np_utils.to_categorical(y_one_hot)

In [8]:
folds = StratifiedKFold(n_splits=10, random_state=2019)
fold_num = 1

for train_idx, val_idx in folds.split(X, y):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_one_hot[train_idx], y_one_hot[val_idx]

    gc.collect()
    K.clear_session()

    model = Sequential()
    model.add(Dense(128, input_shape=(X.shape[1],)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(64))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(32))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=5)
    mcp = ModelCheckpoint(filepath=f'./NN-models/models/fold-{fold_num}.hdf5', 
                          monitor='val_acc', mode='max', 
                          verbose=1, 
                          save_best_only=True)
    
    model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              batch_size=32,
              epochs=50,
              verbose=1,
              callbacks=[mcp, es]
             )
    
    fold_num += 1






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2869 samples, validate on 321 samples
Epoch 1/50
2869/2869 [==============================] - 5s 2ms/step - loss: 0.8926 - acc: 0.6344 - val_loss: 0.5936 - val_acc: 0.7882

Epoch 00001: val_acc improved from -inf to 0.78816, saving model to ./models/fold-1.hdf5
Epoch 2/50
2869/2869 [==============================] - 1s 305us/step - loss: 0.6365 - acc: 0.7389 - val_loss: 0.4493 - val_acc: 0.8505

Epoch 00002: val_acc improved from 0.78816 to 0.85047, saving model to ./models/fold-1.hdf5
Epoch 3/50
2869/2869 [==============================] - 1s 309us/step - loss: 0.5133 - acc: 0.7947 - val_loss: 0.3739 - val_acc: 0.8754

Epoch 00003: val_acc improved from 0.85047 to 0.87539, saving model to ./models/fold-1.hdf5
Epoch 4/50
2869/2869 [==============================] - 

2872/2872 [==============================] - 1s 308us/step - loss: 0.3888 - acc: 0.8569 - val_loss: 0.3958 - val_acc: 0.8333

Epoch 00007: val_acc did not improve from 0.84906
Epoch 8/50
2872/2872 [==============================] - 1s 312us/step - loss: 0.3798 - acc: 0.8538 - val_loss: 0.3891 - val_acc: 0.8396

Epoch 00008: val_acc did not improve from 0.84906
Epoch 9/50
2872/2872 [==============================] - 1s 309us/step - loss: 0.3771 - acc: 0.8545 - val_loss: 0.3933 - val_acc: 0.8208

Epoch 00009: val_acc did not improve from 0.84906
Epoch 00009: early stopping
Train on 2872 samples, validate on 318 samples
Epoch 1/50
2872/2872 [==============================] - 5s 2ms/step - loss: 0.8952 - acc: 0.6097 - val_loss: 0.6431 - val_acc: 0.7830

Epoch 00001: val_acc improved from -inf to 0.78302, saving model to ./models/fold-7.hdf5
Epoch 2/50
2872/2872 [==============================] - 1s 310us/step - loss: 0.6317 - acc: 0.7403 - val_loss: 0.4754 - val_acc: 0.8428

Epoch 00002: v

2872/2872 [==============================] - 1s 317us/step - loss: 0.3441 - acc: 0.8628 - val_loss: 0.4463 - val_acc: 0.8208

Epoch 00010: val_acc did not improve from 0.83333
Epoch 11/50
2872/2872 [==============================] - 1s 319us/step - loss: 0.3376 - acc: 0.8698 - val_loss: 0.4486 - val_acc: 0.8239

Epoch 00011: val_acc did not improve from 0.83333
Epoch 12/50
2872/2872 [==============================] - 1s 316us/step - loss: 0.3348 - acc: 0.8708 - val_loss: 0.4418 - val_acc: 0.8270

Epoch 00012: val_acc did not improve from 0.83333
Epoch 13/50
2872/2872 [==============================] - 1s 341us/step - loss: 0.3249 - acc: 0.8719 - val_loss: 0.4475 - val_acc: 0.8333

Epoch 00013: val_acc did not improve from 0.83333
Epoch 00013: early stopping


In [32]:
val_prob = np.zeros((len(train_per_user), 3), dtype=np.float32)
pred_prob = np.zeros((len(test_per_user), 3), dtype=np.float32)

folds = StratifiedKFold(n_splits=10, random_state=2019)
fold_num = 1

for train_idx, val_idx in folds.split(X, y):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_one_hot[train_idx], y_one_hot[val_idx]

    gc.collect()
    K.clear_session()
    
    model = models.load_model(f'./NN-models/models/fold-{fold_num}.hdf5')
    val_prob[val_idx] = model.predict(X_val, batch_size=32, verbose=1)
    pred_prob = model.predict(X_test, batch_size=32, verbose=1) / 10
    fold_num += 1

802/802 [==============================] - 0s 42us/step


In [33]:
validation = train_per_user[['user-id', 'class']]
validation['prediction'] = encoder.inverse_transform(np.argmax(val_prob, axis=1))
validation[encoder.classes_[0]] = val_prob[:, 0]
validation[encoder.classes_[1]] = val_prob[:, 1]
validation[encoder.classes_[2]] = val_prob[:, 2]
validation.to_csv('./result-per-user/cnn-val.csv', index=False)
accuracy_score(validation['class'], validation['prediction'])

0.8554858934169279

In [41]:
submission = test_per_user[['user-id', 'class']]
submission['prediction'] = encoder.inverse_transform(np.argmax(pred_prob, axis=1))
submission[encoder.classes_[0]] = pred_prob[:, 0]
submission[encoder.classes_[1]] = pred_prob[:, 1]
submission[encoder.classes_[2]] = pred_prob[:, 2]
submission.to_csv('./result-per-user/cnn-sub.csv', index=False)

In [12]:
remove_cols = ['aha', 'ahah', 'ahaha', 'ahahah', 'ahahaha', 'ahahahaha', 'bahaha', 'bahahaha', 'haha', 'hahah', 'hahaha', 'hahahah', 'hahahaha', 'hahahahaha', 'hahahahahaha']

for w in stop_words:
    if w in train.columns:
        remove_cols.append(w)
        
train_per_user = train_per_user.drop(remove_cols, axis=1)
test_per_user = test_per_user.drop(remove_cols, axis=1)

In [13]:
X = train_per_user.drop(['tweet-id', 'user-id', 'class'], axis=1)
X_test = test_per_user.drop(['tweet-id', 'user-id', 'class'], axis=1)

# mmsc = MinMaxScaler()
# X = mmsc.fit_transform(X)
# X_test = mmsc.transform(X_test)

ss = StandardScaler()
X = ss.fit_transform(X)
X_test = ss.transform(X_test)

y = train_per_user['class']
encoder = LabelEncoder()
encoder.fit(y)
y_one_hot = encoder.transform(y)
y_one_hot = np_utils.to_categorical(y_one_hot)

In [14]:
folds = StratifiedKFold(n_splits=10, random_state=2019)
folds = folds.split(X, y)
fold_num = 1

for train_idx, val_idx in folds:
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_one_hot[train_idx], y_one_hot[val_idx]

    gc.collect()
    K.clear_session()
    
    model = Sequential()
    model.add(Dense(128, input_shape=(X.shape[1],)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(64))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(32))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=5)
    mcp = ModelCheckpoint(filepath=f'./models-remove-stopwords/fold-{fold_num}.hdf5', 
                          monitor='val_acc', mode='max', 
                          verbose=1, 
                          save_best_only=True)
    
    model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              batch_size=32,
              epochs=50,
              verbose=1,
              callbacks=[mcp, es]
             )
    
    fold_num += 1

Train on 2869 samples, validate on 321 samples
Epoch 1/50
2869/2869 [==============================] - 6s 2ms/step - loss: 0.9808 - acc: 0.5451 - val_loss: 0.6598 - val_acc: 0.7788

Epoch 00001: val_acc improved from -inf to 0.77882, saving model to ./models-remove-stopwords/fold-1.hdf5
Epoch 2/50
2869/2869 [==============================] - 1s 306us/step - loss: 0.6660 - acc: 0.7309 - val_loss: 0.4787 - val_acc: 0.8474

Epoch 00002: val_acc improved from 0.77882 to 0.84735, saving model to ./models-remove-stopwords/fold-1.hdf5
Epoch 3/50
2869/2869 [==============================] - 1s 321us/step - loss: 0.5490 - acc: 0.7863 - val_loss: 0.3994 - val_acc: 0.8723

Epoch 00003: val_acc improved from 0.84735 to 0.87227, saving model to ./models-remove-stopwords/fold-1.hdf5
Epoch 4/50
2869/2869 [==============================] - 1s 318us/step - loss: 0.5028 - acc: 0.8006 - val_loss: 0.3696 - val_acc: 0.8754

Epoch 00004: val_acc improved from 0.87227 to 0.87539, saving model to ./models-rem

2870/2870 [==============================] - 1s 258us/step - loss: 0.4029 - acc: 0.8422 - val_loss: 0.4477 - val_acc: 0.8500

Epoch 00007: val_acc did not improve from 0.85000
Epoch 8/50
2870/2870 [==============================] - 1s 257us/step - loss: 0.4116 - acc: 0.8387 - val_loss: 0.4524 - val_acc: 0.8500

Epoch 00008: val_acc did not improve from 0.85000
Epoch 9/50
2870/2870 [==============================] - 1s 262us/step - loss: 0.3897 - acc: 0.8509 - val_loss: 0.4460 - val_acc: 0.8469

Epoch 00009: val_acc did not improve from 0.85000
Epoch 10/50
2870/2870 [==============================] - 1s 260us/step - loss: 0.3743 - acc: 0.8533 - val_loss: 0.4460 - val_acc: 0.8531

Epoch 00010: val_acc improved from 0.85000 to 0.85313, saving model to ./models-remove-stopwords/fold-4.hdf5
Epoch 11/50
2870/2870 [==============================] - 1s 260us/step - loss: 0.3688 - acc: 0.8477 - val_loss: 0.4461 - val_acc: 0.8406

Epoch 00011: val_acc did not improve from 0.85313
Epoch 12/50
287


Epoch 00005: val_acc improved from 0.85220 to 0.86478, saving model to ./models-remove-stopwords/fold-7.hdf5
Epoch 6/50
2872/2872 [==============================] - 1s 259us/step - loss: 0.4295 - acc: 0.8276 - val_loss: 0.3850 - val_acc: 0.8648

Epoch 00006: val_acc did not improve from 0.86478
Epoch 7/50
2872/2872 [==============================] - 1s 258us/step - loss: 0.4134 - acc: 0.8339 - val_loss: 0.3791 - val_acc: 0.8679

Epoch 00007: val_acc improved from 0.86478 to 0.86792, saving model to ./models-remove-stopwords/fold-7.hdf5
Epoch 8/50
2872/2872 [==============================] - 1s 257us/step - loss: 0.3945 - acc: 0.8426 - val_loss: 0.3753 - val_acc: 0.8711

Epoch 00008: val_acc improved from 0.86792 to 0.87107, saving model to ./models-remove-stopwords/fold-7.hdf5
Epoch 9/50
2872/2872 [==============================] - 1s 258us/step - loss: 0.3882 - acc: 0.8475 - val_loss: 0.3822 - val_acc: 0.8679

Epoch 00009: val_acc did not improve from 0.87107
Epoch 10/50
2872/2872 [=

Train on 2872 samples, validate on 318 samples
Epoch 1/50
2872/2872 [==============================] - 4s 1ms/step - loss: 0.9983 - acc: 0.5310 - val_loss: 0.6772 - val_acc: 0.7799

Epoch 00001: val_acc improved from -inf to 0.77987, saving model to ./models-remove-stopwords/fold-10.hdf5
Epoch 2/50
2872/2872 [==============================] - 1s 259us/step - loss: 0.6302 - acc: 0.7552 - val_loss: 0.5292 - val_acc: 0.8050

Epoch 00002: val_acc improved from 0.77987 to 0.80503, saving model to ./models-remove-stopwords/fold-10.hdf5
Epoch 3/50
2872/2872 [==============================] - 1s 260us/step - loss: 0.5320 - acc: 0.7890 - val_loss: 0.4726 - val_acc: 0.8145

Epoch 00003: val_acc improved from 0.80503 to 0.81447, saving model to ./models-remove-stopwords/fold-10.hdf5
Epoch 4/50
2872/2872 [==============================] - 1s 258us/step - loss: 0.4710 - acc: 0.8148 - val_loss: 0.4626 - val_acc: 0.8239

Epoch 00004: val_acc improved from 0.81447 to 0.82390, saving model to ./models-

In [15]:
val_prob = np.zeros((len(train_per_user), 3), dtype=np.float32)
pred_prob = np.zeros((len(test_per_user), 3), dtype=np.float32)

folds = StratifiedKFold(n_splits=10, random_state=2019)
fold_num = 1

for train_idx, val_idx in folds.split(X, y):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_one_hot[train_idx], y_one_hot[val_idx]

    gc.collect()
    K.clear_session()
    
    model = models.load_model(f'./models-remove-stopwords/fold-{fold_num}.hdf5')
    val_prob[val_idx] = model.predict(X_val, batch_size=32, verbose=1)
    pred_prob = model.predict(X_test, batch_size=32, verbose=1) / 10
    fold_num += 1

802/802 [==============================] - 0s 35us/step


In [16]:
validation = train_per_user[['user-id', 'class']]
validation['prediction'] = encoder.inverse_transform(np.argmax(val_prob, axis=1))
validation[encoder.classes_[0]] = val_prob[:, 0]
validation[encoder.classes_[1]] = val_prob[:, 1]
validation[encoder.classes_[2]] = val_prob[:, 2]
validation.to_csv('./result-per-user/cnn-remove-stopwords-val.csv', index=False)
accuracy_score(validation['class'], validation['prediction'])

0.8479623824451411

In [17]:
submission = test_per_user[['user-id', 'class']]
submission['prediction'] = encoder.inverse_transform(np.argmax(pred_prob, axis=1))
submission[encoder.classes_[0]] = pred_prob[:, 0]
submission[encoder.classes_[1]] = pred_prob[:, 1]
submission[encoder.classes_[2]] = pred_prob[:, 2]
submission.to_csv('./result-per-user/cnn-remove-stopwords-sub.csv', index=False)